In [14]:
# https://www.youtube.com/watch?v=jmXp9EEsFl0&list=PL_Ke9hJMFeR9Kos7VtiPPiPp9Y16sA7Ef&index=47

1) Random Policy

In [1]:
import gym
import random

env = gym.make('CartPole-v1')

def Random_fn():
    for episode in range(20):
        env.reset()
        for i in range(50):
            ## env.render()
            action = env.action_space.sample()
            
            next_state, reward, done, info = env.step(action)
            print(i, next_state, reward, done, action)
            if done:
                break
Random_fn()
env.close()

AttributeError: module 'numpy' has no attribute 'bool8'

2. SARSA

In [3]:
import gym
import numpy as np
import math
import matplotlib.pyplot as plt

In [8]:
env = gym.make("CartPole-v1")
print(env.observation_space.high)
print(env.observation_space.low)
print(env.action_space.n)

#Hyperparameters
EPISODES = 10
DISCOUNT = 0.95
EPISODE_DISPLAY = 50
LR = 0.25
EPSILON = 0.2

#Q table = state_size * actions spaces * state size
theta_minmax = env.observation_space.high[2]
theta_dot_minmax = math.radians(50)
theta_state_size = 50
theta_dot_state_size  = 50
Q_table = np.random.randn(theta_state_size, theta_dot_state_size, env.action_space.n)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
2


In [5]:
#States
ep_rewards = []
ep_rewards_table = {'ep': [], 'avg': [], 'min':[], 'max': []}

In [10]:
def discretised_state(state):
    discrete_state = np.array([0,0])
    theta_window = ( theta_minmax - (- theta_minmax)) / theta_state_size
    discrete_state[0] = ( state[2] - (- theta_minmax))// theta_window
    discrete_state[0] = min( theta_state_size-1, max(0, discrete_state[0]))
    
    theta_dot_window = ( theta_dot_minmax - (- theta_dot_minmax)) / theta_dot_state_size
    discrete_state[1] = ( state[3] - (- theta_dot_minmax))// theta_dot_window
    discrete_state[1] = min( theta_dot_state_size-1, max(0, discrete_state[1]))
    
    return tuple(discrete_state.astype(int))

In [11]:
for episode in range(EPISODES):
    episode_reward = 0
    done = False
    
    if episode % EPISODE_DISPLAY == 0:
        render_state = True
    else:
        render_state = False
    
    curr_discrete_state = discretised_state(env.reset()[0])
    if np.random.random() > EPSILON:
        action =  np.argmax(Q_table[curr_discrete_state])
    else:
        action = np.random.randint(0, env.action_space.n)
        
    while not done:
        next_state, reward, done, _ = env.step(action)
        new_discrete_state = discretised_state(next_state)
        
        if np.random.random() > EPSILON:
            new_action =  np.argmax(Q_table[new_discrete_state])
        else:
            action = np.random.randint(0, env.action_space.n)

        if render_state:
            env.render()
            
        if not done:
            current_q = Q_table[curr_discrete_state+(action,)]
            max_future_q = Q_table[new_discrete_state+(new_action,)]
            new_q = current_q + LR*(reward+DISCOUNT*max_future_q-current_q)
            Q_table[curr_discrete_state+(action,)] = new_q
        
        curr_discrete_state = new_discrete_state
        action = new_action
        
        episode_reward += reward
    ep_rewards.append(episode_reward)
    
    if not episode % EPISODE_DISPLAY:
        avg_reward = sum(ep_rewards[-EPISODE_DISPLAY:])/len(ep_rewards[-EPISODE_DISPLAY:])
        ep_rewards_table['ep'].append(episode)
        ep_rewards_table['avg'].append(avg_reward)
        ep_rewards_table['min'].append(min(ep_rewards[-EPISODE_DISPLAY:]))
        ep_rewards_table['max'].append(max(ep_rewards[-EPISODE_DISPLAY:]))
        print(f"Episode;{episode} avg:{avg_reward} min:{min(ep_rewards[-EPISODE_DISPLAY:])} max:{max(ep_rewards[-EPISODE_DISPLAY:])}")
        

AttributeError: module 'numpy' has no attribute 'bool8'

In [ ]:
        
env.close()

plt.plot(ep_rewards_table['ep'], ep_rewards_table['avg'], label='avg')
plt.plot(ep_rewards_table['ep'], ep_rewards_table['min'], label='min')
plt.plot(ep_rewards_table['ep'], ep_rewards_table['max'], label='max')
plt.legend(loc=4)
plt.title('CartPole SARSA')
plt.ylabel('Average rewars/episode')
plt.xlabel('Episodes')
plt.show()

3. REINFORCE

In [ ]:
import gym
from tqdm import tqdm_notebook
import numpy as np
from collections import deque

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

ModuleNotFoundError: No module named 'torch'

In [13]:
DISCOUNT_FACTOR = 0.9
NUM_EPISODES = 10
MAX_STEPS = 10
SOLVED_SCORE = 200

DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'

NameError: name 'torch' is not defined

In [ ]:
class PolicyNetwrok(nn.Module):
    def __init__(self, observation_space, action_space):
        super(PolicyNetwrok, self).__init__()
        self.input_layer = nn.Linear(observation_space, 128)
        self.output_layer = nn.Linear(128, action_space)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        actions = self.output_layer(x)
        action_probs = F.softmax(actions, dim=1)
        return action_probs

In [ ]:
class StateValueNetwrok(nn.Module):
    def __init__(self, observation_space):
        super(StateValueNetwrok, self).__init__()
        self.input_layer = nn.Linear(observation_space, 128)
        self.output_layer = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        state_value = self.output_layer(x)
        return state_value

In [ ]:
def select_Action(network, state):
    state = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
    action_probs = network(state)
    state = state.detach()
    
    m = Categorical(action_probs)
    action = m.sample()
    
    return action.item(), m.log_prob(action)

In [ ]:
#Define ENV
env = gym.make('CartPole-v1')

#Inititalize Network
policy_network = PolicyNetwrok(env.observation_space.shape[0], env.action_space.n).to(DEVICE)
state_val_network = StateValueNetwrok(env.observation_space.shape[0]).to(DEVICE)
#Initialize Optimizer
policy_opt = optim.SGD(policy_network.parameters(), lr=0.001)
stateval_opt = optim.SGD(state_val_network.parameters(), lr=0.001)

In [ ]:
scores = []
recent_scores = deque(maxlen=100)
for episode in tqdm_notebook(range(NUM_EPISODES)):
    state = env.reset()
    done = False
    score = 0
    I = 1
    
    #UPDATE ONLINE
    for step in range(MAX_STEPS):
        action, lp = select_Action(policy_network, state) 
        next_state, reward, done, _ = env.step(action)
        score += reward
        
        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
        state_val = state_val_network(state_tensor)
        
        new_tensor_state = torch.from_numpy(next_state).float().unsqueeze(0).to(DEVICE)
        new_state_val = state_val_network(new_tensor_state)
        
        if done:
            new_state_val = torch.tensor([0]).float().unsqueeze(0).to(DEVICE)

        val_loss = F.mse_loss(reward + DISCOUNT_FACTOR * new_state_val , state_val)
        val_loss *= I

        advantage = reward + DISCOUNT_FACTOR * new_state_val.item() - state_val.item()
        policy_loss = -lp * advantage
        policy_loss *= I
        
        policy_opt.zero_grad()
        policy_loss.backward(retain_graph = True)
        policy_opt.step()

        stateval_opt.zero_grad()
        val_loss.backward(retain_graph = True)
        stateval_opt.step()
        
        if done:
            break
            
        state = next_state
        I *= DISCOUNT_FACTOR

    scores.append(score)
    recent_scores.append(score)
    
    if np.array(recent_scores).mean()>= SOLVED_SCORE:
        break

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import numpy as np

sns.set()
plt.plot(scores)
plt.ylabel('score')
plt.xlabel('episodes')
plt.title('Training score of CartPole')


reg = LinearRegression().fit(np.arange(len(scores)).reshape(-1,1),
                             np.array(scores).reshape(-1, 1))
y_pred = reg.predict(np.arange(len(scores)).reshape(-1,1))
plt.plot(y_pred)
plt.show()

In [ ]:
np.mean(scores)

4. DQN

In [15]:
import gym
import random
import numpy as np
import tensorflow as tf

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.95
        self.epsilon = 1.0
        self.min_epsilon = 0.01
        self.episilon_decay = 0.995
        self.learning_rate = 0.001
        self.memory = []
        self.model = self._build_model()
        
    def _build_model(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(24, input_dim=self.state_size, activation='relu'),
            tf.keras.layers.Dense(24, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr = self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.min_epsilon:
            self.epsilon *= self.episilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
if __name__ == '__main__':
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    episodes = 1000
    batch_size = 32
    for r in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state =  next_state
            if done:
                print('Episodes: {}/{}, score: {}, e: {:.2}'.format(r, episodes, time, agent.epsilon))
            if len(agent.memory)>batch_size:
                agent.replay(batch_size)
            if r % 50 == 0:
                agent.save('Cartpole_dqn_model.h5')